# ဟိုတယ်အော်ဒါလုပ်ခြင်းနှင့် Priority Member Middleware

ဒီ notebook က Microsoft Agent Framework ကို အသုံးပြုပြီး **function-based middleware** ကို ပြသထားပါတယ်။ Conditional workflow နမူနာကို အခြေခံပြီး Priority Member တွေကို အထူးအခွင့်အရေးပေးတဲ့ middleware layer ကို ထည့်သွင်းထားပါတယ်။

## သင်လေ့လာရမယ့်အရာများ:
1. **Function-Based Middleware**: Function ရလဒ်များကို ဖမ်းဆီးပြီး ပြင်ဆင်ခြင်း
2. **Context Access**: `context.result` ကို အလုပ်ပြီးဆုံးပြီးနောက် ဖတ်ရှုခြင်းနှင့် ပြင်ဆင်ခြင်း
3. **Business Logic Implementation**: Priority Member အကျိုးခံစားခွင့်များ
4. **Result Override**: အသုံးပြုသူအဆင့်အတန်းအပေါ် မူတည်ပြီး Function ရလဒ်များကို ပြောင်းလဲခြင်း
5. **အလုပ်လုပ်ပုံတူညီ၊ ရလဒ်ကွဲပြားမှု**: Middleware အားဖြင့် အပြောင်းအလဲများ

## Middleware ပါဝင်တဲ့ Workflow Architecture:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Conditional Workflow နှင့် မတူညီတဲ့ အချက်အလက်:

**Middleware မပါဝင်ပါက** (14-conditional-workflow.ipynb):
- ပဲရစ်မှာ အခန်းမရှိ → alternative_agent ကို လမ်းညွှန်

**Middleware ပါဝင်ပါက** (ဒီ notebook):
- သာမန်အသုံးပြုသူ + ပဲရစ် → အခန်းမရှိ → alternative_agent ကို လမ်းညွှန်
- Priority Member + ပဲရစ် → 🌟 Middleware က override လုပ်ပြီး! → အခန်းရရှိ → booking_agent ကို လမ်းညွှန်

## လိုအပ်ချက်များ:
- Microsoft Agent Framework ကို ထည့်သွင်းထားရမည်
- Conditional workflows ကို နားလည်ထားရမည် (14-conditional-workflow.ipynb ကို ကြည့်ပါ)
- GitHub token သို့မဟုတ် OpenAI API key
- Middleware patterns ကို အခြေခံနားလည်မှု


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## အဆင့် ၁ - Structured Outputs အတွက် Pydantic Models ကို သတ်မှတ်ပါ

ဒီမော်ဒယ်တွေက **schema** ကို သတ်မှတ်ပေးပြီး agent တွေက ပြန်ပေးမယ့် အချက်အလက်ပုံစံကို ဖော်ပြပါတယ်။ Middleware က availability ရလဒ်ကို ပြောင်းလဲတဲ့အခါကို မှတ်တမ်းတင်ဖို့ `priority_override` field ကို ထည့်သွင်းထားပါတယ်။


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## အဆင့် ၂ - အရေးကြီးသော အဖွဲ့ဝင်များ ဒေတာဘေ့စ်ကို သတ်မှတ်ပါ

ဒီနမူနာအတွက် အရေးကြီးသော အဖွဲ့ဝင်များ ဒေတာဘေ့စ်ကို အတုအယောင်ဖြင့် ဖန်တီးသုံးပါမည်။ ထုတ်လုပ်မှုတွင်တော့ အမှန်တကယ် ဒေတာဘေ့စ် သို့မဟုတ် API ကို ရှာဖွေမေးမြန်းရမည်ဖြစ်သည်။

**အရေးကြီးသော အဖွဲ့ဝင်များ:**
- `alice@example.com` - VIP အဖွဲ့ဝင်
- `bob@example.com` - အထူးအဆင့် အဖွဲ့ဝင်  
- `priority_user` - စမ်းသပ်အကောင့်


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## အဆင့် ၃ - ဟိုတယ်ဘွတ်ကင်ကိရိယာကို ဖန်တီးပါ

အခြေအနေအလိုက်လုပ်ငန်းစဉ်နဲ့ တူတူပဲ၊ ဒါပေမယ့် အခုတော့ middleware က အလယ်မှာ ဖမ်းဆီးသွားပါမယ်!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## အဆင့် ၄: 🌟 Priority Check Middleware ဖန်တီးခြင်း (အဓိကအချက်!)

ဒီဟာက ဒီ notebook ရဲ့ **အဓိကလုပ်ဆောင်ချက်** ဖြစ်ပါတယ်။ Middleware က:

1. **ဖမ်းယူ** hotel_booking function ကိုခေါ်ဆိုမှု
2. `next(context)` ကိုခေါ်ပြီး function ကိုပုံမှန်အတိုင်း **အကောင်အထည်ဖော်**
3. `context.result` ထဲမှာရလဒ်ကို **စစ်ဆေး**
4. အသုံးပြုသူ priority ဖြစ်ပြီးအခန်းမရှိတဲ့အခါမှာရလဒ်ကို **ပြောင်းလဲ**
5. **ပြောင်းလဲထားတဲ့ရလဒ်** ကို agent ဆီပြန်ပေးပို့

**အဓိက Pattern:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## အဆင့် ၅ - လမ်းကြောင်းသတ်မှတ်ရန် အခြေအနေ ဖန်ရှင်များကို သတ်မှတ်ပါ

အခြေအနေ ဖန်ရှင်များသည် အခြေအနေ Workflow နှင့် တူသည် - ၎င်းတို့သည် ဖွဲ့စည်းထားသော အထွေထွေ Output ကို စစ်ဆေးပြီး လမ်းကြောင်းသတ်မှတ်ရန် ဆုံးဖြတ်သည်။


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## အဆင့် ၆ - အထူးပြသမှု Executor တစ်ခုဖန်တီးပါ

အရင်ကအသုံးပြုခဲ့တဲ့ Executor နဲ့တူတူပဲ - နောက်ဆုံး workflow ရလဒ်ကို ပြသပေးပါသည်။


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## အဆင့် ၇: ပတ်ဝန်းကျင်အပြောင်းအလဲများကို Load လုပ်ပါ

LLM client (GitHub Models သို့မဟုတ် OpenAI) ကို ပြင်ဆင်ပါ။


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## အဆင့် ၈ - Middleware ဖြင့် AI အေးဂျင့်များ ဖန်တီးခြင်း

**အဓိက ကွာခြားချက်:** availability_agent ကို ဖန်တီးတဲ့အခါမှာ `middleware` parameter ကို ပေးပို့ရပါတယ်!

ဒါက priority_check_middleware ကို အေးဂျင့်ရဲ့ function invocation pipeline ထဲသို့ ထည့်သွင်းပေးတဲ့ နည်းလမ်းဖြစ်ပါတယ်။


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## အဆင့် ၉: Workflow ကို တည်ဆောက်ပါ

ယခင် workflow ဖွဲ့စည်းပုံအတိုင်းပဲ - ရရှိနိုင်မှုအခြေခံပြီး အခြေအနေအလိုက် လမ်းကြောင်းသတ်မှတ်ခြင်း။


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## အဆင့် ၁၀ - စမ်းသပ်မှုအမှတ် ၁ - ပုံမှန်အသုံးပြုသူ (Override မရှိ) 

ပုံမှန်အသုံးပြုသူတစ်ဦးသည် ပဲရစ်တွင်အခန်းရရန်ကြိုးစားသည် → အခန်းမရှိ → alternative_agent သို့လမ်းညွှန်သည်


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## အဆင့် ၁၁: စမ်းသပ်မှု အမှတ် ၂ - 🌟 အရေးပါ အသုံးပြုသူ ပဲရစ်တွင် (Override ပါ!)

အရေးပါ အဖွဲ့ဝင်တစ်ဦး ပဲရစ်တွင် အခန်းတစ်ခန်းရယူရန် ကြိုးစားသည် → အခန်းများ မရှိသည့် အခြေအနေ → 🌟 Middleware က Override လုပ်သည်! → booking_agent သို့ လမ်းညွှန်သည်

**ဤသည်မှာ middleware ၏ အာဏာကို ပြသသည့် အဓိက နမူနာဖြစ်သည်!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## အဆင့် ၁၂ - စမ်းသပ်မှု အမှု ၃ - အရေးပါ အသုံးပြုသူ (Priority User) စတော့ခ်ဟုမ်းတွင် (ရရှိပြီးသား)

အရေးပါ အသုံးပြုသူသည် စတော့ခ်ဟုမ်းကို ကြိုးစားသည် → အခန်းများ ရရှိနိုင်သည် → Override မလိုအပ်ပါ → booking_agent သို့ လမ်းညွှန်သည်

ဤသည်မှာ middleware သည် လိုအပ်သောအခါတွင်သာ လုပ်ဆောင်သည်ကို ပြသပါသည်!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## အဓိကအချက်များနှင့် Middleware အကြောင်းအရာများ

### ✅ သင်လေ့လာခဲ့တာများ:

#### **1. Function-Based Middleware Pattern**

Middleware သည် async function တစ်ခုကို အသုံးပြု၍ function call များကို အလယ်တန်းမှတဆင့် ရှုပ်ထွေးစေသည်။

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. Context Access နှင့် Result Override**

- `context.function` - ခေါ်ဆိုနေသော function ကို access လုပ်ရန်
- `context.arguments` - function arguments ကို ဖတ်ရန်
- `context.kwargs` - အပို parameters များကို access လုပ်ရန်
- `await next(context)` - function ကို အကောင်အထည်ဖော်ရန်
- `context.result` - function output ကို ဖတ်/ပြင်ရန်

#### **3. Business Logic အကောင်အထည်ဖော်ခြင်း**

Middleware သည် priority member အကျိုးခံစားခွင့်များကို အကောင်အထည်ဖော်သည်။
- **ပုံမှန်အသုံးပြုသူများ**: ပြင်ဆင်မှုမရှိ၊ ပုံမှန် workflow
- **Priority အသုံးပြုသူများ**: "မရရှိနိုင်" → "ရရှိနိုင်" ကို override လုပ်သည်
- **အခြေအနေအလိုက် logic**: လိုအပ်သောအခါတွင်သာ override လုပ်သည်

#### **4. တူညီသော Workflow၊ မတူညီသောရလဒ်များ**

Middleware ၏ အားသာချက်များ:
- ✅ Workflow ဖွဲ့စည်းမှုကို မပြောင်းလဲခြင်း
- ✅ Tool function ကို မပြောင်းလဲခြင်း
- ✅ Conditional routing logic ကို မပြောင်းလဲခြင်း
- ✅ Middleware သာ → အခြားအပြောင်းအလဲများ!

### 🚀 အမှန်တကယ်အသုံးချမှုများ:

1. **VIP/Premium Features**
   - Premium အသုံးပြုသူများအတွက် rate limits ကို override လုပ်သည်
   - အရင်းအမြစ်များကို priority access ပေးသည်
   - Premium features များကို dynamic အနေနှင့် unlock လုပ်သည်

2. **A/B Testing**
   - အသုံးပြုသူများကို အခြား implementation များသို့ route လုပ်သည်
   - အသုံးပြုသူအချို့နှင့် features အသစ်များကို စမ်းသပ်သည်
   - Gradual feature rollouts

3. **Security & Compliance**
   - Function calls များကို audit လုပ်သည်
   - အရေးကြီးသောလုပ်ဆောင်မှုများကို block လုပ်သည်
   - Business rules များကို အကောင်အထည်ဖော်သည်

4. **Performance Optimization**
   - အသုံးပြုသူအချို့အတွက် result များကို cache လုပ်သည်
   - လိုအပ်သောအခါတွင် အကျိုးကျေးဇူးမရှိသောလုပ်ဆောင်မှုများကို ကျော်လွှားသည်
   - Dynamic resource allocation

5. **Error Handling & Retry**
   - Error များကို gracefully handle လုပ်သည်
   - Retry logic ကို အကောင်အထည်ဖော်သည်
   - အခြား implementation များသို့ fallback လုပ်သည်

6. **Logging & Monitoring**
   - Function execution time များကို track လုပ်သည်
   - Parameters နှင့် results များကို log လုပ်သည်
   - Usage patterns များကို monitor လုပ်သည်

### 🔑 Decorators နှင့် Middleware ၏ အဓိကကွာခြားချက်များ:

| Feature | Decorator | Middleware |
|---------|-----------|------------|
| **Scope** | Single function | Agent အတွင်းရှိ function အားလုံး |
| **Flexibility** | Definition အချိန်တွင် fix | Runtime အချိန်တွင် dynamic |
| **Context** | အကန့်အသတ်ရှိ | Full agent context |
| **Composition** | Decorators များစွာ | Middleware pipeline |
| **Agent-Aware** | မရှိ | Agent state ကို access လုပ်နိုင်သည် |

### 📚 Middleware ကို ဘယ်အချိန်တွင် အသုံးပြုမလဲ:

✅ **Middleware ကို အသုံးပြုပါ:**
- User/session state အပေါ်မူတည်၍ အပြုအမူကို ပြောင်းလဲရန်လိုအပ်သောအခါ
- Function များစွာတွင် logic ကို အသုံးပြုလိုသောအခါ
- Agent-level context ကို access လုပ်လိုသောအခါ
- Cross-cutting concerns (logging, auth, စသည်) ကို အကောင်အထည်ဖော်လိုသောအခါ

❌ **Middleware ကို မအသုံးပြုပါ:**
- ရိုးရှင်းသော input validation (Pydantic ကို အသုံးပြုပါ)
- Function-specific logic (function အတွင်းတွင်ထားပါ)
- တစ်ခါတစ်လေပြောင်းလဲမှုများ (function ကို ပြောင်းလဲပါ)

### 🎓 အဆင့်မြင့် Patterns:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 ဆက်စပ်အကြောင်းအရာများ:

- **Agent Middleware**: Agent.run() calls များကို intercept လုပ်သည်
- **Function Middleware**: Tool function calls များကို intercept လုပ်သည် (ကျွန်ုပ်တို့အသုံးပြုခဲ့သည်!)
- **Middleware Pipeline**: Middleware များ၏ အစဉ်အတိုင်း အကောင်အထည်ဖော်မှု
- **Context Propagation**: Middleware chain အတွင်း state ကို pass လုပ်သည်



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွဲအချော်များ သို့မဟုတ် အနားယူမှုမှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
